##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Word embeddings

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/word_embeddings">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/word_embeddings.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/word_embeddings.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/text/word_embeddings.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial contains an introduction to word embeddings. You will train your own word embeddings using a simple Keras model for a sentiment classification task, and then visualize them in the [Embedding Projector](http://projector.tensorflow.org) (shown in the image below). 

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding.jpg?raw=1" alt="Screenshot of the embedding projector" width="400"/>

## Representing text as numbers

Machine learning models take vectors (arrays of numbers) as input. When working with text, the first thing you must do is come up with a strategy to convert strings to numbers (or to "vectorize" the text) before feeding it to the model. In this section, you will look at three strategies for doing so.

### One-hot encodings

As a first idea, you might "one-hot" encode each word in your vocabulary. Consider the sentence "The cat sat on the mat". The vocabulary (or unique words) in this sentence is (cat, mat, on, sat, the). To represent each word, you will create a zero vector with length equal to the vocabulary, then place a one in the index that corresponds to the word. This approach is shown in the following diagram.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/one-hot.png?raw=1" alt="Diagram of one-hot encodings" width="400" />

To create a vector that contains the encoding of the sentence, you could then concatenate the one-hot vectors for each word.

**Key point: This approach is inefficient.** A one-hot encoded vector is sparse (meaning, most indices are zero). Imagine you have 10,000 words in the vocabulary. To one-hot encode each word, you would create a vector where 99.99% of the elements are zero.

### Encode each word with a unique number

A second approach you might try is to encode each word using a **unique number**. Continuing the example above, you could assign 1 to "cat", 2 to "mat", and so on. You could then encode the sentence "The cat sat on the mat" as a dense vector like [5, 1, 4, 3, 5, 2]. This appoach is efficient. Instead of a sparse vector, you now have a dense one (where all elements are full).

There are two downsides to this approach, however:

* The integer-encoding is arbitrary (it does not capture any relationship between words).

* An integer-encoding can be challenging for a model to interpret. A linear classifier, for example, learns a single weight for each feature. Because there is no relationship between the similarity of any two words and the similarity of their encodings, this feature-weight combination is not meaningful.

### Word embeddings

Word embeddings give us a way to use an efficient, dense representation in which similar words have a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a dense vector of floating point values (the length of the vector is a parameter you specify). Instead of specifying the values for the embedding manually, they are trainable parameters (weights learned by the model during training, in the same way a model learns weights for a dense layer). It is common to see word embeddings that are 8-dimensional (for small datasets), up to 1024-dimensions when working with large datasets. A higher dimensional embedding can capture fine-grained relationships between words, but takes more data to learn.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding2.png?raw=1" alt="Diagram of an embedding" width="400"/>

Above is a diagram for a word embedding. Each word is represented as a 4-dimensional vector of floating point values. Another way to think of an embedding is as "lookup table". After these weights have been learned, you can encode each word by looking up the dense vector it corresponds to in the table.

## Setup

In [2]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

### Download the IMDb Dataset
You will use the [Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) through the tutorial. You will train a sentiment classifier model on this dataset and in the process learn embeddings from scratch. To read more about loading a dataset from scratch, see the [Loading text tutorial](../load_data/text.ipynb).  

Download the dataset using Keras file utility and take a look at the directories.

In [3]:
'''
Cuidado NO subir luego al GitHub
Acceder mejor a la URL y descargar directamente en carpeta Descargas

url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')
'''
#dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
#os.listdir(dataset_dir)

'\nCuidado NO subir luego al GitHub\nAcceder mejor a la URL y descargar directamente en carpeta Descargas\n\nurl = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"\n\ndataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,\n                                  untar=True, cache_dir=\'.\',\n                                  cache_subdir=\'\')\n'

Take a look at the `train/` directory. It has `pos` and `neg` folders with movie reviews labelled as positive and negative respectively. You will use reviews from `pos` and `neg` folders to train a binary classification model.

In [3]:
dataset_dir = "C:\\Users\\alber\\Downloads\\aclImdb_v1\\aclImdb\\train"
#train_dir = os.path.join(dataset_dir, 'train')
#os.listdir(train_dir)

The `train` directory also has additional folders which should be removed before creating training dataset.

In [3]:
#remove_dir = os.path.join(train_dir, 'unsup')
#shutil.rmtree(remove_dir)

Next, create a `tf.data.Dataset` using `tf.keras.preprocessing.text_dataset_from_directory`. You can read more about using this utility in this [text classification tutorial](https://www.tensorflow.org/tutorials/keras/text_classification). 

Use the `train` directory to create both train and validation datasets with a split of 20% for validation.

In [4]:
batch_size = 1024
seed = 123

'''
Busca en un directorio todas las carpetas. Cada carpeta es una etiqueta
Y cada archivo una review. Podemos especificar si es para el subset
de training o validation y cuanto dejamos para validacion.
Esto crea un tf.data.Dataset
ELIMINAR UNA CARPETA EN TRAIN, QUE SOBRA
'''

train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    dataset_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    dataset_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation', # Esto y la semilla permiten que las muestras con train no se superpongan
    seed=seed)

Found 75000 files belonging to 3 classes.
Using 60000 files for training.
Found 75000 files belonging to 3 classes.
Using 15000 files for validation.


Take a look at a few movie reviews and their labels `(1: positive, 0: negative)` from the train dataset.


In [5]:
train_ds.take(10)

<TakeDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [6]:
for text_batch, label_batch in train_ds.take(1): # Cogemos el primer batch
  for i in range(5): # 1025 da error xq no hay mas en este batch
    print(label_batch[i].numpy(), text_batch.numpy()[i])

1 b"Ask yourself where she got the gun? Remember what she was taught about the mark's mindset when the con is over? The gun had blanks and it was provided to her from the very beginning.<br /><br />When the patient comes back at the end she was SUPPOSED to see him drive away in the red convertible and lead her to the gang splitting up her 80 thousand.<br /><br />The patient was in on the con from the beginning.<br /><br />Mantegna does not die in the end - the gun had blanks.<br /><br />There - enough spoilers for you there? This is why people are giving it such high ratings. It's extremely original because of the hidden ending and how it cons MOST of the audience."
2 b"For some reason, people seem to have a problem differentiating this movie from Trail of the Pink Panther.<br /><br />At any rate, this work does nothing but serve to remind us how sad the world is without Peter Sellers in it.<br /><br />They brought back the same old favorites from Trail (Dreyfus, Cato, Litton, etc.), b

### Configure the dataset for performance

These are two important methods you should use when loading data to make sure that I/O does not become blocking.

`.cache()` keeps data in memory after it's loaded off disk. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache, which is more efficient to read than many small files.

`.prefetch()` overlaps data preprocessing and model execution while training. 

You can learn more about both methods, as well as how to cache data to disk in the [data performance guide](https://www.tensorflow.org/guide/data_performance).

In [7]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Using the Embedding layer

Keras makes it easy to use word embeddings. Take a look at the [Embedding](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer.

The Embedding layer can be understood as a lookup table that maps from integer indices (which stand for specific words) to dense vectors (their embeddings). The dimensionality (or width) of the embedding is a parameter you can experiment with to see what works well for your problem, much in the same way you would experiment with the number of neurons in a Dense layer.


In [8]:
'''
Embedd a 1,000 word vocabulary into 5 dimensions.
Razonar cantidad del vocabulario y embeddings con el cap de "AI and ML for coders"
https://learning.oreilly.com/library/view/ai-and-machine/9781492078180/ch06.html

La capa de embedding funciona como un diccionario al que asigna cada palabra con su vector de embedding
Estos vectores se entrenan con backpropagation

Basicamente embeddings funciona bien como la raiz cuarta del vocabulario
Y el vocabulario esta bien tener palabras con al menos 20 como frecuencia
de aparición
'''
embedding_layer = tf.keras.layers.Embedding(1000, 5)

When you create an Embedding layer, the weights for the embedding are randomly initialized (just like any other layer). During training, they are gradually adjusted via backpropagation. Once trained, the learned word embeddings will roughly encode similarities between words (as they were learned for the specific problem your model is trained on).

If you pass an integer to an embedding layer, the result replaces each integer with the vector from the embedding table:

In [9]:
tf.constant([1, 2, 999])

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([  1,   2, 999])>

In [10]:
'''
No puede hacer mas de la palabra 1000
Se inicializa el embedding con pesos aleatorios
1000 daria error ya que no hay tanto vocab
5 valores para cada "palabra" numérica
'''
result = embedding_layer(tf.constant([1, 2, 999]))
result.numpy()

array([[ 0.03042907, -0.00772778,  0.03136179,  0.02621094,  0.0139096 ],
       [-0.00522513, -0.01752614, -0.02593738,  0.00014941, -0.03875433],
       [ 0.01231589,  0.0234954 , -0.02520455,  0.00919622,  0.00337904]],
      dtype=float32)

In [11]:
print(embedding_layer.embeddings.shape)
embedding_layer.embeddings

(1000, 5)


<tf.Variable 'embedding/embeddings:0' shape=(1000, 5) dtype=float32, numpy=
array([[ 0.02576594, -0.0064076 ,  0.02969674, -0.0350096 ,  0.01755558],
       [ 0.03042907, -0.00772778,  0.03136179,  0.02621094,  0.0139096 ],
       [-0.00522513, -0.01752614, -0.02593738,  0.00014941, -0.03875433],
       ...,
       [ 0.00392149,  0.03900056, -0.03911978,  0.03802966, -0.00448322],
       [ 0.00909553, -0.02326909,  0.0353284 ,  0.04282594, -0.0194979 ],
       [ 0.01231589,  0.0234954 , -0.02520455,  0.00919622,  0.00337904]],
      dtype=float32)>

For text or sequence problems, the Embedding layer takes a 2D tensor of integers, of shape `(samples, sequence_length)`, where each entry is a sequence of integers. It can embed sequences of variable lengths. You could feed into the embedding layer above batches with shapes `(32, 10)` (batch of 32 sequences of length 10) or `(64, 15)` (batch of 64 sequences of length 15).

The returned tensor has one more axis than the input, the embedding vectors are aligned along the new last axis. Pass it a `(2, 3)` input batch and the output is `(2, 3, N)`


In [11]:
result = embedding_layer(tf.constant([[0, 1, 2],
                                      [3, 4, 5]]))
print(result.shape)
result.numpy()

(2, 3, 5)


array([[[ 0.01236947, -0.01822526,  0.00336619,  0.01731164,
         -0.02586402],
        [ 0.00603884, -0.03774611, -0.04411054,  0.03116176,
          0.02258295],
        [-0.03226171, -0.0435437 ,  0.03335794, -0.0366528 ,
          0.02722783]],

       [[-0.0183346 , -0.01161654,  0.00477195,  0.03667328,
          0.01383214],
        [-0.0063717 ,  0.04064864, -0.00227129,  0.00501939,
         -0.00971254],
        [ 0.04645033, -0.046709  ,  0.02918023, -0.00826325,
         -0.04302992]]], dtype=float32)

When given a batch of sequences as input, an embedding layer returns a 3D floating point tensor, of shape `(samples, sequence_length, embedding_dimensionality)`. To convert from this sequence of variable length to a fixed representation there are a variety of standard approaches. You could use an RNN, Attention, or pooling layer before passing it to a Dense layer. This tutorial uses pooling because it's the simplest. The [Text Classification with an RNN](text_classification_rnn.ipynb) tutorial is a good next step.

## Text preprocessing

Next, define the dataset preprocessing steps required for your sentiment classification model. Initialize a TextVectorization layer with the desired parameters to vectorize movie reviews. You can learn more about using this layer in the [Text Classification](https://www.tensorflow.org/tutorials/keras/text_classification) tutorial.

In [12]:
# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 10000 # Ojo, al final es 10k y no 1k
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
# Nos quitamos los labels de esta manera, que van en conjunto con features en train_ds
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


## Create a classification model

Use the [Keras Sequential API](https://www.tensorflow.org/guide/keras/sequential_model) to define the sentiment classification model. In this case it is a "Continuous bag of words" style model.
* The [`TextVectorization`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/TextVectorization) layer transforms strings into vocabulary indices. You have already initialized `vectorize_layer` as a TextVectorization layer and built it's vocabulary by calling `adapt` on `text_ds`. Now vectorize_layer can be used as the first layer of your end-to-end classification model, feeding transformed strings into the Embedding layer.
* The [`Embedding`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer takes the integer-encoded vocabulary and looks up the embedding vector for each word-index. These vectors are learned as the model trains. The vectors add a dimension to the output array. The resulting dimensions are: `(batch, sequence, embedding)`.

* The [`GlobalAveragePooling1D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalAveragePooling1D) layer returns a fixed-length output vector for each example by averaging over the sequence dimension. This allows the model to handle input of variable length, in the simplest way possible.

* The fixed-length output vector is piped through a fully-connected ([`Dense`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)) layer with 16 hidden units.

* The last layer is densely connected with a single output node. 

Caution: This model doesn't use masking, so the zero-padding is used as part of the input and hence the padding length may affect the output.  To fix this, see the [masking and padding guide](https://www.tensorflow.org/guide/keras/masking_and_padding).

In [40]:
embedding_dim=16

'''
GlobalAveragePooling1D
Cada palabra tiene asociado un embedding. El ouput es la media de cada
coordenada del embedding, por tanto, si hay 16 embeddings, hará un 
flatten a 16, siendo cada valor la media de la coordenada de ese 
embedding para todas las palabras de la review
'''

model = Sequential([
  vectorize_layer, # 100 [1, 3, 4, 4, 90, ...]
  Embedding(vocab_size, embedding_dim, name="embedding"), # 10.000 x 16 --> [[], [], [] ...] 100x16
  GlobalAveragePooling1D(), # [] 100
  Dense(16, activation='relu'), # 
  Dense(1, activation='sigmoid') # originalmente no tiene activacion
])

## Compile and train the model

You will use [TensorBoard](https://www.tensorflow.org/tensorboard) to visualize metrics including loss and accuracy. Create a `tf.keras.callbacks.TensorBoard`.

In [41]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

Compile and train the model using the `Adam` optimizer and `BinaryCrossentropy` loss. 

In [42]:
model.compile(optimizer='adam',
              # binary_crossentropy
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [43]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
20/20 [==============================] - 38s 2s/step - loss: 0.6926 - accuracy: 0.5495 - val_loss: 0.6901 - val_accuracy: 0.6618
Epoch 2/15
20/20 [==============================] - 3s 133ms/step - loss: 0.6887 - accuracy: 0.6917 - val_loss: 0.6839 - val_accuracy: 0.7052
Epoch 3/15
20/20 [==============================] - 3s 133ms/step - loss: 0.6814 - accuracy: 0.7035 - val_loss: 0.6732 - val_accuracy: 0.7128
Epoch 4/15
20/20 [==============================] - 3s 137ms/step - loss: 0.6691 - accuracy: 0.7172 - val_loss: 0.6567 - val_accuracy: 0.7264
Epoch 5/15
20/20 [==============================] - 3s 134ms/step - loss: 0.6496 - accuracy: 0.7363 - val_loss: 0.6334 - val_accuracy: 0.7406
Epoch 6/15
20/20 [==============================] - 3s 133ms/step - loss: 0.6227 - accuracy: 0.7552 - val_loss: 0.6050 - val_accuracy: 0.7532
Epoch 7/15
20/20 [==============================] - 3s 134ms/step - loss: 0.5900 - accuracy: 0.7773 - val_loss: 0.5737 - val_accuracy: 0.7676
Epoch 8/

With this approach the model reaches a validation accuracy of around 84% (note that the model is overfitting since training accuracy is higher).

Note: Your results may be a bit different, depending on how weights were randomly initialized before training the embedding layer. 

You can look into the model summary to learn more about each layer of the model.

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


Visualize the model metrics in TensorBoard.

In [18]:
#docs_infra: no_execute
%load_ext tensorboard
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 1764), started 1 day, 22:35:36 ago. (Use '!kill 1764' to kill it.)

![embeddings_classifier_accuracy.png](images/embeddings_classifier_accuracy.png)

## Retrieve the trained word embeddings and save them to disk

Next, retrieve the word embeddings learned during training. The embeddings are weights of the Embedding layer in the model. The weights matrix is of shape `(vocab_size, embedding_dimension)`.

Obtain the weights from the model using `get_layer()` and `get_weights()`. The `get_vocabulary()` function provides the vocabulary to build a metadata file with one token per line. 

In [19]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

Write the weights to disk. To use the [Embedding Projector](http://projector.tensorflow.org), you will upload two files in tab separated format: a file of vectors (containing the embedding), and a file of meta data (containing the words).

In [20]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

If you are running this tutorial in [Colaboratory](https://colab.research.google.com), you can use the following snippet to download these files to your local machine (or use the file browser, *View -> Table of contents -> File browser*).

In [21]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

## Visualize the embeddings

To visualize the embeddings, upload them to the embedding projector.

Open the [Embedding Projector](http://projector.tensorflow.org/) (this can also run in a local TensorBoard instance).

* Click on "Load data".

* Upload the two files you created above: `vecs.tsv` and `meta.tsv`.

The embeddings you have trained will now be displayed. You can search for words to find their closest neighbors. For example, try searching for "beautiful". You may see neighbors like "wonderful". 

Note: Experimentally, you may be able to produce more interpretable embeddings by using a simpler model. Try deleting the `Dense(16)` layer, retraining the model, and visualizing the embeddings again.

Note: Typically, a much larger dataset is needed to train more interpretable word embeddings. This tutorial uses a small IMDb dataset for the purpose of demonstration.


## Next Steps

This tutorial has shown you how to train and visualize word embeddings from scratch on a small dataset.

* To train word embeddings using Word2Vec algorithm, try the [Word2Vec](https://www.tensorflow.org/tutorials/text/word2vec) tutorial. 

* To learn more about advanced text processing, read the [Transformer model for language understanding](https://www.tensorflow.org/tutorials/text/transformer).